In [ ]:
"""Imports"""
import ipywidgets

In [ ]:
"""Globale Variablen, Konstanten und Setup-Parameter"""
out = ipywidgets.Output(layout={'border': '1px solid black'})
layout = ipywidgets.GridspecLayout(7, 7, width="250px")
debug = False # Skip "Setzphase"
setup = True # Disables "Setzphase" automatically
turn_player_one = True # Which player's turn
setupcounter = 0 # Counter for amount stones set in "Setzphase"
selection = None # Stone which will be moved on the nect button click
muehle = False # Enables a turn in which a player removes a stone
FELDER = ((0,0),(0,3),(0,6),(1,1),(1,3),(1,5),(2,2),(2,3),(2,4),(3,0),(3,1),(3,2),(3,4),(3,5),(3,6),(4,2),(4,3),(4,4),(5,1),(5,3),(5,5),(6,0),(6,3),(6,6)) # Valid "Mühle" fields in a 7x7 grid
MUEHLEN = [[(0,0),(0,3),(0,6),False], # Configurations of possible "Mühlen" and a boolean to indicate whether a player achieved it
           [(1,1),(1,3),(1,5),False],
           [(2,2),(2,3),(2,4),False],
           [(3,0),(3,1),(3,2),False],
           [(3,4),(3,5),(3,6),False],
           [(4,2),(4,3),(4,4),False],
           [(5,1),(5,3),(5,5),False],
           [(6,0),(6,3),(6,6),False],
           [(0,0),(3,0),(6,0),False],
           [(1,1),(3,1),(5,1),False],
           [(2,2),(3,2),(4,2),False],
           [(0,3),(1,3),(2,3),False],
           [(4,3),(5,3),(6,3),False],
           [(2,4),(3,4),(4,4),False],
           [(1,5),(3,5),(5,5),False],
           [(0,6),(3,6),(6,6),False]]

In [ ]:
"""Function definitions"""
def player_color():
    """Return player name of current turn as color."""
    global turn_player_one
    return "gelb" if turn_player_one else "Schwarz"

@out.capture()
def check_muehle():
    """Durchlaufen Sie mögliche „Mühle“-Konfigurationen und initiieren Sie ggf. „Mühle-Turn“.
    neue 'Mühle' gefunden."""
    global layout
    global muehle
    global MUEHLEN
    global turn_player_one
    
    player = "yellow" if turn_player_one else "black"
    for i, muehlen_tuple in enumerate(MUEHLEN):
        color1 = layout[muehlen_tuple[0][0],muehlen_tuple[0][1]].style.button_color
        color2 = layout[muehlen_tuple[1][0],muehlen_tuple[1][1]].style.button_color
        color3 = layout[muehlen_tuple[2][0],muehlen_tuple[2][1]].style.button_color

        if player == color1 and color1 == color2 and color2 == color3 and not muehlen_tuple[3]:
            # New "Mühle" found
            MUEHLEN[i][3] = True
            muehle = True
            out.clear_output()
            print(f"Mühle: {muehlen_tuple[:3]}. {player_color()} entfernt einen Stein.")

        if color1 != color2 or color2 != color3:
            # Remove old "Mühlen"
            MUEHLEN[i][3] = False

@out.capture()
def turn(self):
    """Kernfunktion für die Spiellogik. Wird jedes Mal aufgerufen, wenn auf eine Schaltfläche geklickt wird.
    Unterscheidet zwischen „Setzphase“ und „Zugphase“ und fügt eine zusätzliche Runde ein, wenn eine neue „Mühle“ erkannt wird.
    In der Zugphase unterscheidet diese Funktion auch zwischen der Auswahl des zu verschiebenden Steins und dem eigentlichen Zug.
    Außerdem wird festgehalten, welcher Spieler gerade an der Reihe ist."""
    global setup
    global turn_player_one
    global setupcounter
    global selection
    global debug
    global muehle

    out.clear_output()
    if muehle:
        # Additional phase to remove a stone
        if self.style.button_color == "#8f4c2b":
            # Keep game state if empty field was selected
            print(f"Mühle {player_color()}")
            return
        self.style.button_color = "#8f4c2b"
        self.style.text_color = "#8f4c2b"
        muehle = False
        turn_player_one = not turn_player_one
        print(f"{player_color()} am Zug.")
    elif setup and not debug:
        # "Setzphase", skipped in debug mode
        if self.style.button_color != "#8f4c2b":
            # Keep game state if empty field was selected
            print(f"{player_color()} am Zug.")
            return
        if turn_player_one:
            self.style.button_color = "yellow"
            self.style.text_color = "yellow"
        else:
            self.style.button_color = "black"
            self.style.text_color = "black"
        setupcounter += 1
        if setupcounter == 18:
            # End "Setzphase" after each player set 9 stones
            setup = False
            print(f"Setzphase beendet. Zugphase beginnt.")
        check_muehle()
        if not muehle:
            turn_player_one = not turn_player_one
            print(f"{player_color()} am Zug.")
    else:
        # "Zugphase"
        if selection:
            # Move selected stone
            if self.style.button_color != "#8f4c2b":
                # Keep game state if occupied field was selected
                print(f"{player_color()} am Zug.")
                return
            self.style.button_color = selection.style.button_color
            self.style.text_color = selection.style.text_color
            selection.style.button_color = "#8f4c2b"
            selection.style.text_color = "#8f4c2b"
            selection = None
            check_muehle()
            if not muehle:
                turn_player_one = not turn_player_one
                print(f"{player_color()} am Zug.")
        else:
            # Select stone to be moved on next button click
            if self.style.button_color == "#8f4c2b":
                print(f"{player_color()} am Zug.")
                return
            selection = self
            print(f"{self.description} ausgewählt.")

In [ ]:
"""Erstellen Sie eine Tafel als 7x7-Raster und platzieren Sie 24 Felder als Schaltflächen auf den gültigen „Mühle“-Positionen.
Wir verwenden zwei verschiedene Positionsargumente. Ein kartesisches System für die Position im Raster und
ein Polarsystem, bei dem das erste Argument die Ringnummer und das zweite Argument die Position auf dem Ring ist.
Das Polarkoordinatensystem erleichtert das Auffinden benachbarter Felder, obwohl diese Funktion in dieser Version nicht implementiert ist.
Das für das Polarsystem verwendete Attribut ist die Beschreibung der Schaltfläche."""
FELDER= ((0,0),(0,3),(0,6),(1,1),(1,3),(1,5),(2,2),(2,3),(2,4),(3,0),(3,1),(3,2),(3,4),(3,5),(3,6),(4,2),(4,3),(4,4),(5,1),(5,3),(5,5),(6,0),(6,3),(6,6))
colors = ["yellow","black"]
c = 0
desc = ""
fieldcounter = 0
tempcounter = 0

for i in range(7):
    for j in range(7):
        if (i, j) in FELDER:
            # Valid "Mühle" field
            if i < 3:
                # First 3 board rows
                desc = f"{i},{fieldcounter%3}"
                fieldcounter += 1
            elif i == 3:
                # Middle board row
                if j < 3:
                    desc = f"{j},7"
                else:
                    desc = f"{6-j},3"
            else:
                # Last 3 board rows
                fieldcounter = 6
                desc = f"{6-i},{fieldcounter-tempcounter%3}"
                tempcounter += 1
                    
            layout[i,j] = ipywidgets.Button(description=desc, layout=ipywidgets.Layout(width='30px'))
            layout[i,j].style.button_color = "#8f4c2b"
            layout[i,j].style.text_color = "#8f4c2b"
            if debug and c < 18:
                # Automatically place stones on board in debug mode
                layout[i,j].style.text_color = colors[c%2]
                layout[i,j].style.button_color = colors[c%2]
            layout[i,j].on_click(turn) 
            c += 1
        else:
            # Not a "Mühle" field and therefore simple plain board
            layout[i,j] = ipywidgets.Button(description="", layout=ipywidgets.Layout(width='30px'))
            layout[i,j].disabled = True

In [ ]:
display(out, layout)

In [ ]:
""" Wenn es keine Schwarze Steine hat, merkt das Spiel nicht das es zu Ende ist """
""" es hat keine feste Regel ausser das man die belegte Plätze nicht tauschen kann"""
"""jeder muss die Regel selber festlegen und befolgen"""